In [11]:
# %matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact


In [12]:

%matplotlib inline


In [13]:
# Define the ordered list of scale degree labels.
scale_degrees = [
    "A2", "dd3", "AA7", "dd-2", "m2", "P8", "A6", "AAA4", "d2", "m7",
    "A3", "d3", "A7", "dd5", "d5", "d4", "AA8", "M2", "m6", "AA2",
    "M7", "dd4", "m3", "A5", "AA5", "A4", "AA3", "ddd5", "M3", "P5",
    "d7", "P4", "AA1", "dd7", "AA6", "A1", "M6", "AA4", "A8", "d6",
    "dd2", "dd6"
]
n_scale = len(scale_degrees)

In [14]:
def parse_graph_from_lines(lines):
    """
    Given a list of nonempty lines for one graph, parse and return (N, nodes, matrix).
    
    The expected format is:
       N=50
       X:
       <node values on one or more lines>
       E:
       <edge matrix values spanning N*N numbers across one or more lines>
    """
    # First line is "N=50" (or similar)
    N = int(lines[0].split("=")[1])
    
    # Find the indices for the "X:" and "E:" markers.
    try:
        x_index = lines.index("X:") + 1
    except ValueError:
        raise ValueError("Could not find 'X:' section in the graph chunk.")
    try:
        e_index = lines.index("E:")
    except ValueError:
        raise ValueError("Could not find 'E:' section in the graph chunk.")
    
    # Gather all node values from lines between "X:" and "E:".
    x_values = []
    for line in lines[x_index:e_index]:
        x_values.extend(line.split())
    if len(x_values) != N:
        raise ValueError(f"Expected {N} node values but found {len(x_values)}")
    nodes = [int(val) for val in x_values]
    
    # Gather all edge matrix numbers from the lines after "E:".
    edge_numbers = []
    for line in lines[e_index+1:]:
        edge_numbers.extend(line.split())
    if len(edge_numbers) != N * N:
        raise ValueError(f"Expected {N*N} edge values but found {len(edge_numbers)}")
    
    # Build the N x N matrix.
    matrix = []
    for i in range(N):
        row = [int(edge_numbers[i * N + j]) for j in range(N)]
        matrix.append(row)
    
    return N, nodes, matrix

In [15]:
def parse_graphs_file(filename):
    """
    Read the file and split it into separate graph chunks.
    Each graph is assumed to begin with a line starting with "N=".
    Returns a list of tuples (N, nodes, matrix) for each graph.
    """
    with open(filename, "r") as f:
        # Read and strip blank lines.
        lines = [line.strip() for line in f if line.strip()]
    
    graphs_data = []
    i = 0
    while i < len(lines):
        if lines[i].startswith("N="):
            # Start of a new graph.
            current_chunk = []
            # Collect lines until the next "N=" or end-of-file.
            while i < len(lines) and not (lines[i].startswith("N=") and current_chunk):
                current_chunk.append(lines[i])
                i += 1
            # If the next graph starts immediately, current_chunk will contain one complete graph.
            graphs_data.append(parse_graph_from_lines(current_chunk))
        else:
            i += 1
    return graphs_data


In [16]:

def build_graph(N, node_vals, matrix):
    """
    Build a directed graph from node values and the edge matrix.
    A node value of 51 is treated as a rest.
    """
    G = nx.DiGraph()
    for i, val in enumerate(node_vals):
        if val == 51:
            label = "Rest"
        else:
            label = scale_degrees[val % n_scale]
        # Adding the node with its label (optionally include the node index).
        G.add_node(i, label=label)
    
    # Add edges only if the weight is nonzero.
    for i in range(N):
        for j in range(N):
            w = matrix[i][j]
            if w != 0:
                G.add_edge(i, j, weight=w)
    return G


In [17]:

def visualize_graph(G):
    """
    Visualize the directed graph using a spring layout.
    Node labels and edge weight labels are shown.
    """
    pos = nx.spring_layout(G)
    labels = {n: f"{G.nodes[n]['label']}\n({n})" for n in G.nodes()}
    
    nx.draw_networkx_nodes(G, pos, node_color="lightblue", node_size=500)
    nx.draw_networkx_labels(G, pos, labels, font_size=8)
    nx.draw_networkx_edges(G, pos, arrowstyle='->', arrowsize=10, edge_color='gray')
    
    # Optionally display edge weight labels.
    edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', font_size=6)
    
    plt.axis("off")
    plt.tight_layout()
    plt.show()


In [18]:
# --- Main interactive section ---

# Specify the path to your file containing one or more graphs.
filename = "generated_samples1.txt"
# Parse the file into a list of graph tuples.
graphs_data = parse_graphs_file(filename)

In [19]:

def update_graph(graph_index, threshold):
    """
    Update the graph visualization.
    Selects a graph by its index and filters out edges with weight < threshold.
    """
    N, node_vals, matrix = graphs_data[graph_index]
    G = build_graph(N, node_vals, matrix)
    
    # Create a filtered graph that only includes edges meeting the threshold.
    H = nx.DiGraph()
    for node, data in G.nodes(data=True):
         H.add_node(node, **data)
    for u, v, data in G.edges(data=True):
         if data['weight'] >= threshold:
             H.add_edge(u, v, **data)
    
    plt.figure(figsize=(10, 8))
    visualize_graph(H)


In [20]:

# Create widgets: a dropdown for selecting the graph and a slider for edge weight threshold.
graph_dropdown = widgets.Dropdown(
    options=[(f"Graph {i}", i) for i in range(len(graphs_data))],
    description="Graph:"
)
threshold_slider = widgets.IntSlider(
    value=0, min=0, max=30, step=1, description="Min Edge Weight:"
)



In [21]:

interact(update_graph, graph_index=graph_dropdown, threshold=threshold_slider)

interactive(children=(Dropdown(description='Graph:', options=(('Graph 0', 0), ('Graph 1', 1), ('Graph 2', 2)),…

<function __main__.update_graph(graph_index, threshold)>